# One Shot

Ziel ist es, mit einem Request alle Antworten zu erhalten.

In [ ]:
import sqlite3
import response_format
from dotenv import load_dotenv
from litellm import completion

load_dotenv()

connection = sqlite3.connect('../data/survey.db')
cursor = connection.cursor()

In [ ]:
# 

In [ ]:
reponse = completion(
    model = "mistral/mistral-tiny",
    messages = [
        {"role": "user", "content": "Antworte mit genau einer Zahl zwischen 1 und 4"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "survey_response",
            "schema": response_format.CompleteResponseFormat.model_json_schema()
        }
    }
)